# Rendmi

Datadump used updated untl 06/07/2022

Source: http://dati.isprambiente.it/accesso-ai-dati/download/

In order to analyze tyhe dataset, gather all the data needed and turn it into a .json file we will use rdflib to query the graph.

In [11]:
# all imports
import pprint
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import XSD, RDFS, DCTERMS
from rdflib import Literal

Data had some errors that needed to be manualy corrected in order to maake the parser work.

**Specifically**:

Invalid lines sucH as *-schema#label>"[ sometext ]".*  were changed into *-schema#label> "[ sometext ]".* 

In [12]:
# create an empty Graph
g = rdflib.Graph()

# parse a local RDF file by specifying the format into the graph
result = g.parse("../datasets/originals/dissesto_07_06_2022.nt", format='nt')

Looks abit weird as 50 entries are missing w.r.t. the original .nt file

Next step consists in subsetting thedataset nd keeping oly facts relted to Emilia romagna.

Thus we previously gathered a [list](http://dati.isprambiente.it/sparql?default-graph-uri=&query=select+distinct+%3Fp+%3Fplabel%0D%0Awhere+%7B%0D%0A%3Fs+%3Fp+%3Fo.%0D%0A%3Fp+rdfs%3Alabel+%3Fplabel.%0D%0Afilter%28lang%28%3Fplabel%29+%3D+%27en%27%29%7D%0D%0A&format=text%2Fhtml&timeout=0&debug=on) of all predicates in the dataset and individuated http://www.geonames.org/ontology#parentADM1 as a startng point, as it is the predicate referring to the italian region 

In [23]:
#look at regions

types = set()

type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
instability = '<http://dati.isprambiente.it/ontology/core#Instability>'

for s, p, o in result.triples((None, type, None)):
    types.add(o)

print(types)

{rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#EconomicIndicator'), rdflib.term.URIRef('http://purl.org/goodrelations/v1#BusinessEntity'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#LotStep'), rdflib.term.URIRef('http://purl.org/procurement/public-contracts#FrameworkAgreement'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Contract'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#ConceptScheme'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Lithology'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Intervention'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#AuthorityKind'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Instability'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Repair')}


In [18]:
#look at regions

types = set()

type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
instability = '<http://dati.isprambiente.it/ontology/core#Instability>'

for s, p, o in result.triples((None, type, None)):
    types.add(o)

print(types)

set()